In [1]:
import random
from random import randint
import string
import faker
from faker import Faker 
import datetime
from datetime import datetime,timedelta
import csv

In [211]:
f = Faker()

In [212]:
#random date between two dates

def r_date(s,e):
    return datetime(s.year,s.month,s.day) + timedelta(days=random.randint(0,(e-s).days))

In [213]:
#r_date(datetime(2018,1,1),datetime(2022,12,31))

In [214]:
def customer_id(length=10):
    characters = string.ascii_uppercase + string.digits
    return ''.join(random.choices(characters,k=length))

In [215]:
#generate customer data

#from Singapore
def customer_singapore():
    country = "Singapore"
    return {
        "Customer_id":f.uuid4(),
        "First_Name":f.first_name(),
        "Last_Name":f.last_name(),
        "Email":f.email(),
        "Phone No.":f.phone_number(),
        "Country_of_Residence": country,
        "Age": random.randint(18,70)
    }

In [216]:
#from Asia
def customer_asia():
    country = random.choices(["China","India","Indonesia","Japan","Thailand","UK","Malaysia","Philippines",
                             "Australia","USA","Vietnam","New Zealand","France","Italy","Spain"],weights=[3,9,3,8,6,9,4,5,7,10,1,1,1,1,1],k=1)[0]
     
    #country = f.random_element(elements=country_weights)
 
    return {
        "Customer_id":f.uuid4(),
        "First_Name":f.first_name(),
        "Last_Name":f.last_name(),
        "Email":f.email(),
        "Phone No.":f.phone_number(),
        "Country_of_Residence": country,
        "Age": random.randint(18,70)
    }

In [217]:
#from anywhere
def customer():
    return {
        "Customer_id":f.uuid4(),
        "First_Name":f.first_name(),
        "Last_Name":f.last_name(),
        "Email":f.email(),
        "Phone No.":f.phone_number(),
        "Country_of_Residence": f.country(),
        "Age": random.randint(18,70)
    }

In [218]:
#generate the reservation data
def reservation(customer_id):
    #num_rooms = random.randint(1,5)
    
    num_people = random.randint(1,3)
    
    method_of_outreach = random.choices(["Websearch","Instagram","Facebook","TikTok","Email","Google Ads"],weights=[3,2,1,2,1,0.5],k=1)[0]
    #take note to change the date and the # of days for that year
    checkin_date = r_date(datetime(2022,1,1),datetime(2022,1,31))
    
    checkout_date = checkin_date + timedelta(days=randint(1,10))
    
    #reservation can be made 2 months in advance from the checkin date
    reservation_date = r_date(checkin_date - timedelta(days=60),checkin_date)
    
    return {
        "Reservation_id":f.uuid4(),
        "Customer_id": customer_id,
        "Number_of_people": num_people,
        "Method_of_Outreach": method_of_outreach,
        "CheckIn_Date": checkin_date,
        "CheckOut_Date": checkout_date,
        "Reservation_Date": reservation_date,
    }
    
    

In [219]:
#generate room ID

def room_id():
    return ''.join(random.choices(string.ascii_lowercase + string.digits,k=8))

#reservation details entity
def reservation_detail(reservation_id):
    #add some weights to ensure that expensive rooms are selected less
    #SD-Standard,DB-Double,DL-Delux,ST-Suite
    room_type = random.choices(["SD","DB","DL","ST"],weights=[3,2,1,0.5],k=1)[0]
    price_per_night = 0
    if room_type == "SD":
        price_per_night = random.randint(100,200)
    elif room_type == "DB":
        price_per_night = random.randint(150,250)
    elif room_type == "DL":
        price_per_night = random.randint(200,300)
    elif room_type == "ST":
        price_per_night = random.randint(250,350)
        
    return {
        "Reservation_id": reservation_id,
        "Room_id":room_id(),
        "Room_Type":room_type,
        "Price_Per_Night":price_per_night
    }

In [220]:
#print(reservation_detail(33333))

In [221]:
#this one only run once because it is meant to include all the sum of the 12 months in the year
#customers_all = []
#reservations_all = []
#reservation_details_all = []
#payment_details_all = []

In [222]:
customers = []
n = int(193*31*0.6) #need to change the days of the year accordingly and also the % of reservations booked
for i in range(n):
    if i <= n*0.35: #out of these reservations booked need to change based on the % of it from SG
        customers.append(customer_singapore())
    else:
        customers.append(customer_asia())


In [223]:
reservations = []
reservation_details = []
payment_details = []

In [224]:
#payment detail entity
def payment_detail(reservation_id,reservation_date,checkout_date,total_amount):
    payment_method = random.choices(["AMEX","VISA","MasterCard","Debit Card","PayNow","Cash","GrabPay"],
                                    weights=[1.1,1.65,3.7,0.5,0.75,0.05,1.15],k=1)[0]
    payment_date = r_date(reservation_date,checkout_date)
    return {
        "Payment_id": f.uuid4(),
        "Reservation_id":reservation_id,
        "Total Amount":total_amount,
        "Date":payment_date,
        "Method": payment_method
    }

In [225]:
for i in customers:
    #b = random.choices([1,2],weights=[7,3],k=1)[0]
    for j in range(1):
        reservation_data = reservation(i["Customer_id"])
        reservations.append(reservation_data)
        total_amount = 0
        
        for k in range(1):
            reservation_room = reservation_detail(reservation_data["Reservation_id"])
            reservation_details.append(reservation_room)
            total_amount = total_amount + reservation_room["Price_Per_Night"]*1.08
        payment_details.append(payment_detail(reservation_data["Reservation_id"],reservation_data["Reservation_Date"],
                                              reservation_data["CheckOut_Date"],total_amount))
        

In [226]:
print(len(reservation_details))

3589


In [227]:
customers_all.extend(customers)
reservations_all.extend(reservations)
reservation_details_all.extend(reservation_details)
payment_details_all.extend(payment_details)

In [228]:
def write_to_csv(data, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        for item in data:
            writer.writerow(item)

In [229]:
write_to_csv(customers_all, 'C:/Users/AN000/OneDrive/Desktop/Masters Program/Trimester 2 (13 Nov - 24 Feb 2024)/MH6341 - Data Management & Business [1.5 AU]/Team Project (Due 24 Feb)/customers_2022.csv')
write_to_csv(reservations_all, 'C:/Users/AN000/OneDrive/Desktop/Masters Program/Trimester 2 (13 Nov - 24 Feb 2024)/MH6341 - Data Management & Business [1.5 AU]/Team Project (Due 24 Feb)/reservations_2022.csv')
write_to_csv(reservation_details_all, 'C:/Users/AN000/OneDrive/Desktop/Masters Program/Trimester 2 (13 Nov - 24 Feb 2024)/MH6341 - Data Management & Business [1.5 AU]/Team Project (Due 24 Feb)/reservation_details_2022.csv')
write_to_csv(payment_details_all, 'C:/Users/AN000/OneDrive/Desktop/Masters Program/Trimester 2 (13 Nov - 24 Feb 2024)/MH6341 - Data Management & Business [1.5 AU]/Team Project (Due 24 Feb)/payment_details_2022.csv')